# RNA-Seq Pipeline: Step 3 - Alignment (HISAT2)

This notebook performs the alignment step. We map our cleaned, paired-end reads (from `02_Trimmed_Data`) to the reference genome (from `06_Genome_Index`).

**Workflow:**
1.  **Build Index:** First, create a `HISAT2` index from our reference genome (`genome.fna.gz`). This is a one-time step that makes alignment fast.
2.  **Run Alignment:** Loop through all 6 samples and run `hisat2` to align the paired-end reads.
3.  **Convert & Sort:** Convert the output (`.sam`) to the compressed `.bam` format (which saves space) and sort it.
4.  **Index BAM:** Create an index (`.bai`) for the sorted BAM files. This is required for visualization and counting.

**Tools:** `hisat2`, `samtools` (which is part of the `subread` package we installed)

In [4]:
import os

# --- Define Core Paths ---

# 1. Input: Cleaned FASTQ files
trimmed_dir = "02_Trimmed_Data"

# 2. Input: Reference Genome files
genome_dir = "06_Genome_Index"
# --- [THE NEW FIX] ---
os.makedirs(genome_dir, exist_ok=True) # <-- 
# --- [END FIX] ---

# --- [THE OLD FIX (Rule 4)] ---
# (hisat2-build needs the *unzipped* file, so we fix the variable)
genome_fasta = os.path.join(genome_dir, "genome.fna") # <-- REMOVED .gz
# --- [END FIX] ---

# 3. Output: HISAT2 Index
index_dir = "07_HISAT2_Index"
os.makedirs(index_dir, exist_ok=True)
index_prefix = os.path.join(index_dir, "klebsiella_hs11286_index")

# 4. Output: Aligned BAM files
alignment_dir = "03_Aligned_BAMs"
os.makedirs(alignment_dir, exist_ok=True)

# --- Get Sample Names ---
input_files = sorted([f for f in os.listdir(trimmed_dir) if f.endswith(".trimmed_R1.fastq.gz")])
sample_names = [f.split(".trimmed_R1.fastq.gz")[0] for f in input_files]

print(f"Genome FASTA: {genome_fasta}") 
print(f"Index Prefix: {index_prefix}")
print(f"Alignment Output Dir: {alignment_dir}")
print(f"Found {len(sample_names)} samples to align: {sample_names}")

Genome FASTA: 06_Genome_Index/genome.fna
Index Prefix: 07_HISAT2_Index/klebsiella_hs11286_index
Alignment Output Dir: 03_Aligned_BAMs
Found 6 samples to align: ['SRR34134104', 'SRR34134105', 'SRR34134106', 'SRR34134107', 'SRR34134108', 'SRR34134109']


### 1: Download Reference Genome 

**Goal:** Download the Reference Genome and Annotation file for *K. pneumoniae* HS11286.

**Why:** This step was *missing* from the original `Notebook 01`. We cannot build an index (Step 3.1) without the FASTA (`.fna.gz`) file. We will download the official Assembly files (ASM24018v2) that match our SRA data.

**Tool:** `wget`

In [5]:
# --- 0. [THE NEW FIX] Clean the directory first ---
print(f"--- Cleaning target directory ---")
!rm -f {genome_dir}/*
print("Directory cleaned.")

# --- 1. Define *Local/Temporary* Paths for this cell ---
# (We use "genome_fasta_GZ" so we DON'T overwrite the main "genome_fasta" variable from Cell 1)
genome_fasta_GZ = os.path.join(genome_dir, "genome.fna.gz") # Local variable
genome_gff_GZ = os.path.join(genome_dir, "genome.gff.gz") # Local variable

# --- 2. Define the URLs (The *CORRECT* V2 assembly) ---
fasta_url = "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/240/185/GCA_000240185.2_ASM24018v2/GCA_000240185.2_ASM24018v2_genomic.fna.gz"
gff_url = "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/240/185/GCA_000240185.2_ASM24018v2/GCA_000240185.2_ASM24018v2_genomic.gff.gz"

# --- 3. Run the Download ---
print(f"\n--- Downloading FASTA (The Genome) ---")
!wget -O {genome_fasta_GZ} {fasta_url}

print(f"\n--- Downloading GFF (The Annotation) ---")
!wget -O {genome_gff_GZ} {gff_url}

# --- 4. Decompress the FASTA for hisat2-build ---
print(f"\n--- Decompressing FASTA ({genome_fasta_GZ}) ---")
!gunzip {genome_fasta_GZ} # This will now work 100%

print("\n--- Download & Decompression complete. Verifying files: ---")
# (We should now *only* see "genome.fna" (UNZIPPED) and "genome.gff.gz")
!ls -lh {genome_dir}

--- Cleaning target directory ---
Directory cleaned.

--- Downloading FASTA (The Genome) ---
--2025-11-04 17:36:28--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/240/185/GCA_000240185.2_ASM24018v2/GCA_000240185.2_ASM24018v2_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.31, 130.14.250.10, 130.14.250.11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1679373 (1.6M) [application/x-gzip]
Saving to: ‘06_Genome_Index/genome.fna.gz’

06_Genome_Index/gen 100%[===================>]   1.60M  3.31MB/s    in 0.5s    

2025-11-04 17:36:29 (3.31 MB/s) - ‘06_Genome_Index/genome.fna.gz’ saved [1679373/1679373]


--- Downloading GFF (The Annotation) ---
--2025-11-04 17:36:29--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/240/185/GCA_000240185.2_ASM24018v2/GCA_000240185.2_ASM24018v2_genomic.gff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 13

In [6]:
%%bash -s "$genome_fasta" "$index_prefix"
# $1 = genome_fasta (from Cell 2)
# $2 = index_prefix (from Cell 2)

GENOME_FASTA=$1
INDEX_PREFIX=$2

echo "--- 1. Building HISAT2 Index ---"
echo "Input FASTA: $GENOME_FASTA"
echo "Output Prefix: $INDEX_PREFIX"

# -p 8 : Use 8 threads
hisat2-build -p 8 $GENOME_FASTA $INDEX_PREFIX

echo "--- HISAT2 Index build complete. ---"
echo "Index files are in: $(dirname $INDEX_PREFIX)"

--- 1. Building HISAT2 Index ---
Input FASTA: 06_Genome_Index/genome.fna
Output Prefix: 07_HISAT2_Index/klebsiella_hs11286_index


Settings:
  Output files: "07_HISAT2_Index/klebsiella_hs11286_index.*.ht2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Local offset rate: 3 (one in 8)
  Local fTable chars: 6
  Local sequence length: 57344
  Local sequence overlap between two consecutive indexes: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  06_Genome_Index/genome.fna
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
  Time to read SNPs and splice sites: 00:00:00
Using parameters --bmax 133179 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 133179 --dcv 102

Building DifferenceCoverSample
  Building sPrime
  Building sPrimeOrder
  V-Sorting samples
  V-Sorting samples time: 00:00:00
  Allocating rank array
  Ranking v-sort output
  Ranking v-sort output time: 00:00:00
  Invoking Larsson-Sadakane on ranks
  Invoking Larsson-Sadakane on ranks time: 00:00:00
  Sanity-checking and returning
Building samples
Reserving space for 86 sample suffixes
Generating random suffixes
QSorting 86 sample offsets, eliminating duplicates
QSorting sample offsets, eliminating duplicates time: 00:00:00
Multikey QSorting 86 samples
  (Using difference cover)
  Multikey QSorting samples time: 00:00:00
Calculating bucket sizes
Splitting and merging
  Splitting and merging time: 00:00:00
Split 12, merged 38; iterating...
Splitting and merging
  Splitting and merging time: 00:00:00
Split 7, merged 5; iterating...
Splitting and merging
  Splitting and merging time: 00:00:00
Split 2, merged 4; iterating...
Splitting and merging
  Splitting and merging time: 00:00:00
Sp

Converting suffix-array elements to index image
Allocating ftab, absorbFtab
Entering GFM loop


Getting block 1 of 60
  Reserving size (133179) for bucket 1
  Calculating Z arrays for bucket 1
  Entering block accumulator loop for bucket 1:
Getting block 2 of 60
  Reserving size (133179) for bucket 2
  Calculating Z arrays for bucket 2
Getting block 3 of 60
  Reserving size (133179) for bucket 3
  Entering block accumulator loop for bucket 2:
Getting block 4 of 60
  Reserving size (133179) for bucket 4
  Calculating Z arrays for bucket 3
Getting block 5 of 60
  Reserving size (133179) for bucket 5
  Calculating Z arrays for bucket 5
  Calculating Z arrays for bucket 4
  Entering block accumulator loop for bucket 3:
Getting block 6 of 60
  Reserving size (133179) for bucket 6
  Entering block accumulator loop for bucket 4:
  Entering block accumulator loop for bucket 5:
  Calculating Z arrays for bucket 6
Getting block 7 of 60
  Reserving size (133179) for bucket 7
  Entering block accumulator loop for bucket 6:
Getting block 8 of 60
  Reserving size (133179) for bucket 8
  Calcul

Exited GFM loop
fchr[A]: 0
fchr[C]: 1219661
fchr[G]: 2843006
fchr[T]: 4465490
fchr[$]: 5682321
Exiting GFM::buildToDisk()
Returning from initFromVector
Wrote 6089371 bytes to primary GFM file: 07_HISAT2_Index/klebsiella_hs11286_index.1.ht2
Wrote 1420588 bytes to secondary GFM file: 07_HISAT2_Index/klebsiella_hs11286_index.2.ht2
Re-opening _in1 and _in2 as input streams
Returning from GFM constructor
Returning from initFromVector
Wrote 2526315 bytes to primary GFM file: 07_HISAT2_Index/klebsiella_hs11286_index.5.ht2
Wrote 1445872 bytes to secondary GFM file: 07_HISAT2_Index/klebsiella_hs11286_index.6.ht2
Re-opening _in5 and _in5 as input streams
Returning from HGFM constructor
Headers:
    len: 5682321
    gbwtLen: 5682322
    nodes: 5682322
    sz: 1420581
    gbwtSz: 1420581
    lineRate: 6
    offRate: 4
    offMask: 0xfffffff0
    ftabChars: 10
    eftabLen: 0
    eftabSz: 0
    ftabLen: 1048577
    ftabSz: 4194308
    offsLen: 355146
    offsSz: 1420584
    lineSz: 64
    sideSz: 6

--- HISAT2 Index build complete. ---
Index files are in: 07_HISAT2_Index


### 2. Run Alignment & BAM Processing Loop

Now that the genome index is built (step 1), we will loop through all 6 samples.

This code cell performs the main alignment workflow:
1.  **Alignment (`hisat2`):** Align the trimmed `R1` and `R2` files to the `klebsiella_hs11286_index`. This outputs a very large text file (`.sam`).
2.  **Conversion (`samtools view`):** Convert the text `.sam` file into its binary (compressed) equivalent (`.bam`).
3.  **Sorting (`samtools sort`):** Sort the `.bam` file by genomic coordinates. This is **required** for the next step (Counting).
4.  **Indexing (`samtools index`):** Create an index file (`.bai`) for the sorted BAM, allowing tools to access it quickly.
5.  **Cleanup (`rm`):** Delete the intermediate `.sam` file to save disk space.

In [7]:
# Cell 3: Run the Alignment Loop 

print("\n--- Starting HISAT2 Alignment Loop ---")

for sample in sample_names:
    print(f"Processing sample: {sample} ...")
    
    # 1. Define inputs
    in_r1 = f"{trimmed_dir}/{sample}.trimmed_R1.fastq.gz"
    in_r2 = f"{trimmed_dir}/{sample}.trimmed_R2.fastq.gz"
    
    # 2. Define outputs
    sam_output = f"{alignment_dir}/{sample}.sam"
    bam_output_sorted = f"{alignment_dir}/{sample}.sorted.bam"
    stats_output = f"{alignment_dir}/{sample}.stats.txt"

    # 4. Run hisat2 alignment
    !hisat2 -p 8 \
        -x $index_prefix \
        -1 $in_r1 \
        -2 $in_r2 \
        --summary-file $stats_output \
        -S $sam_output
    
    print(f"  ... Finished alignment (SAM) for {sample}.")

    # 5. --- THIS IS THE FIX ---
    # Convert, Sort, and Index in one simple command
    # samtools sort can read SAM, sort it, and output BAM directly.
    !samtools sort -@ 8 -o $bam_output_sorted $sam_output
    
    print(f"  ... Finished sorting and converting to BAM for {sample}.")

    # 6. Index the new sorted BAM file
    !samtools index $bam_output_sorted
    
    print(f"  ... Finished indexing BAM for {sample}.")

    # 7. (Important) Clean up the large SAM file
    !rm $sam_output
    
    print(f"  ... Cleaned up intermediate SAM file.")
    print(f"  ... Final output: {bam_output_sorted}")
    print("--------------------------------------")

print("--- HISAT2 Alignment complete for all samples. ---")


--- Starting HISAT2 Alignment Loop ---
Processing sample: SRR34134104 ...
20728434 reads; of these:
  20728434 (100.00%) were paired; of these:
    2080132 (10.04%) aligned concordantly 0 times
    17966907 (86.68%) aligned concordantly exactly 1 time
    681395 (3.29%) aligned concordantly >1 times
    ----
    2080132 pairs aligned concordantly 0 times; of these:
      1599220 (76.88%) aligned discordantly 1 time
    ----
    480912 pairs aligned 0 times concordantly or discordantly; of these:
      961824 mates make up the pairs; of these:
        552579 (57.45%) aligned 0 times
        311996 (32.44%) aligned exactly 1 time
        97249 (10.11%) aligned >1 times
98.67% overall alignment rate
  ... Finished alignment (SAM) for SRR34134104.
[bam_sort_core] merging from 2 files and 8 in-memory blocks...
  ... Finished sorting and converting to BAM for SRR34134104.
  ... Finished indexing BAM for SRR34134104.
  ... Cleaned up intermediate SAM file.
  ... Final output: 03_Aligned_BAMs

### 3. Verify Alignment Statistics (MultiQC)

The alignment loop is complete. `hisat2` generated a summary statistics file (`.stats.txt`) for each sample.

We will now run `MultiQC` on the `03_Aligned_BAMs` directory to parse these stats files and create a single report. This report will show us the "Overall Alignment Rate" for all samples, which is the most critical metric to verify success.

In [8]:
# Define the output directory for this new QC report
# (We need 'os' again if the kernel was restarted, but it should be loaded)
import os 

alignment_qc_dir = "00_Data_QC/03_alignment_qc"
os.makedirs(alignment_qc_dir, exist_ok=True)

print(f"Alignment stats (input): {alignment_dir}")
print(f"QC Report (output): {alignment_qc_dir}")

Alignment stats (input): 03_Aligned_BAMs
QC Report (output): 00_Data_QC/03_alignment_qc


In [9]:
%%bash -s "$alignment_dir" "$alignment_qc_dir"
# $1 = alignment_dir (where the .stats.txt files are)
# $2 = alignment_qc_dir (where the report will go)

ALIGN_DIR=$1
QC_OUT_DIR=$2

echo "--- 6. Running MultiQC on Alignment Stats ---"
echo "Scanning directory: $ALIGN_DIR"
echo "Outputting report to: $QC_OUT_DIR"

# -o : output directory
# ALIGN_DIR : target directory to scan
multiqc -o $QC_OUT_DIR $ALIGN_DIR

echo "--- Alignment MultiQC complete. ---"
echo "Check the 'multiqc_report.html' file in $QC_OUT_DIR"

--- 6. Running MultiQC on Alignment Stats ---
Scanning directory: 03_Aligned_BAMs
Outputting report to: 00_Data_QC/03_alignment_qc



/// ]8;id=54473;https://multiqc.info\MultiQC]8;;\ 🔍 v1.30

     version_check | MultiQC Version v1.32 now available!
       file_search | Search path: /home/refm_youssef/rnaseq_klebsiella_pipeline/03_Aligned_BAMs
         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 18/18  18  
           bowtie2 | Found 6 reports
     write_results | Data        : 00_Data_QC/03_alignment_qc/multiqc_data
     write_results | Report      : 00_Data_QC/03_alignment_qc/multiqc_report.html
           multiqc | MultiQC complete


--- Alignment MultiQC complete. ---
Check the 'multiqc_report.html' file in 00_Data_QC/03_alignment_qc


## Conclusion 

**Status:** 100% Success.

**Analysis:**
The R&D for this phase is complete.
1.  **Genome Index:** We successfully downloaded the correct reference genome (`.fna`) and built the `hisat2` index (`.ht2` files).
2.  **Alignment Loop:** The `hisat2 | samtools sort` pipe ran successfully for all 6 samples.
3.  **QC Verification:** The final `MultiQC` report (above) confirms excellent alignment rates (all >97.9%) for the entire cohort.

**Final Product:**
We have successfully produced 6 high-quality, sorted, and indexed BAM files (`.sorted.bam` + `.sorted.bam.bai`) in the `03_Aligned_BAMs/` directory.

**Next Step (The Handoff):**
We are now 100% ready to proceed to the next phase as defined in the `README.md`:
**`04_Read_Counting.ipynb`** (where we will use `featureCounts` to generate the "Counts Matrix").

In [10]:
%%bash
echo "--- Verifying FINAL outputs in 03_Aligned_BAMs ---"

# We expect 18 files:
# 6x .sorted.bam (The Alignments)
# 6x .sorted.bam.bai (The BAM Indexes)
# 6x .stats.txt (The HISAT2 Reports)

ls -lh 03_Aligned_BAMs/

--- Verifying FINAL outputs in 03_Aligned_BAMs ---
total 5.7G
-rw-rw-r-- 1 refm_youssef refm_youssef  1.2G Nov  4 17:45 SRR34134104.sorted.bam
-rw-rw-r-- 1 refm_youssef refm_youssef  110K Nov  4 17:45 SRR34134104.sorted.bam.bai
-rw-rw-r-- 1 refm_youssef refm_youssef   632 Nov  4 17:44 SRR34134104.stats.txt
-rw-rw-r-- 1 refm_youssef refm_youssef  848M Nov  4 17:49 SRR34134105.sorted.bam
-rw-rw-r-- 1 refm_youssef refm_youssef   76K Nov  4 17:49 SRR34134105.sorted.bam.bai
-rw-rw-r-- 1 refm_youssef refm_youssef   632 Nov  4 17:48 SRR34134105.stats.txt
-rw-rw-r-- 1 refm_youssef refm_youssef  874M Nov  4 17:54 SRR34134106.sorted.bam
-rw-rw-r-- 1 refm_youssef refm_youssef  158K Nov  4 17:54 SRR34134106.sorted.bam.bai
-rw-rw-r-- 1 refm_youssef refm_youssef   632 Nov  4 17:53 SRR34134106.stats.txt
-rw-rw-r-- 1 refm_youssef refm_youssef  846M Nov  4 17:59 SRR34134107.sorted.bam
-rw-rw-r-- 1 refm_youssef refm_youssef  222K Nov  4 17:59 SRR34134107.sorted.bam.bai
-rw-rw-r-- 1 refm_youssef refm_you